In [ ]:
import sys                                                                                                                                                                                                              
sys.path[0]='/home/jovyan/mnt/neuronunit'                                                                                                                                                                                                     
from neuronunit.tests import get_neab                                                                                                                                                                                                         
import neuronunit                                                                                                                                                                                                                             
print(neuronunit.tests.__file__)                                                                                                                                                                                                              
from neuronunit.tests import utilities as outils                                                                                                                                                                                              
                                                                                                                                                                                                                                              
from neuronunit.tests import model_parameters as modelp                                                                                                                                                                                       
import numpy as np                                                                                                                                                                                                                            
model = outils.model                
from sciunit import scores


Getting Rheobase cached data value for from AIBS dataset 354190013
attempting to recover from pickled file
/home/jovyan/mnt/neuronunit/neuronunit/tests/__init__.py
Ignoring included LEMS file: Cells.xml
Ignoring included LEMS file: Networks.xml
Ignoring included LEMS file: Simulation.xml
Mechanisms already loaded from path: /home/jovyan/mnt/neuronunit/neuronunit/tests/NeuroML2.  Aborting.
Ignoring included LEMS file: Cells.xml
Ignoring included LEMS file: Networks.xml
Ignoring included LEMS file: Simulation.xml


In [ ]:

class Score:
    def __init__(self, score):
        self.score = score

class Test:
    def _optimize(self, model,modelp):
        '''
        The implementation of optimization, consisting of implementation details.
        Inputs a model, and model parameter ranges to expore
        Private method for programmer designer.
        Outputs the optimal model, its attributes and the low error it resulted in.
        '''
        from neuronunit.tests import nsga
        gap = nsga.GAparams(model)
        # The number of generations is 2
        gap.NGEN = 3
        # The population of genes is 4
        gap.MU = 12
        
        gap.BOUND_LOW = [ np.min(i) for i in modelp.model_params.values() ]
        gap.BOUND_UP = [ np.max(i) for i in modelp.model_params.values() ]

        vmpop, pop, invalid_ind, pf = nsga.main(gap.NGEN,gap.MU,model,modelp)
        some_tuples = (vmpop, pop, invalid_ind, pf)
        return pop[0], some_tuples
        
    def _get_optimization_parameters(self, some_tuples):
        vmpop = some_tuples[0]
        # Your specific unpacking of tuples that _optimize returns
        scores = [ i.score for i in vmpop ]
        pop = some_tuples[1]
        
        errors = pop[0].fitness
        parameters = vmpop[0].attrs
      
        #parameters,errors,scores,_ = zip(*some_tuples)
        return parameters,scores

    def optimize(self, model, modelp):
        '''
        The Class users version of optimize
        where details are hidden in _optimizae
        '''

        # Do optimization including repeated calls to judge
        models, some_tuples = self._optimize(model,modelp)
        parameters, scores = self._get_optimization_parameters(some_tuples)
        # Maybe rebuild the original model
        # (i.e. restore the true class from the virtual version)

        # Your code keeps parameter sets and associated scores
        # All the organizing stuff
        
        
        # this a way of looking at solved model parameters, ie candidate solutions from 
        # the pareto front.

        
        # Make a ScoreArray (which is basically a pandas dataframe)
       
        return model, scores, some_tuples

    
t = Test()


model,scores,some_tuples = t.optimize(model,modelp)   

<module 'neuronunit.tests.model_parameters' from '/home/jovyan/mnt/neuronunit/neuronunit/tests/model_parameters.py'>
0 -73.400262041
1 36.4524842237
2 -58.5434523695
3 0.124862070926
4 -4.40578546097e-09
5 0.179072394484
6 9.72425551651e-05
7 0.0007796795291
8 -44.6032302514
9 -55.9630798427
0 -58.5242551327
1 33.0107157088
2 -72.2325673634
3 0.466450283841
4 -4.98182194127e-09
5 0.146182635518
6 9.18533525657e-05
7 0.000665578583789
8 -41.0093438624
9 -55.4635958265
0 -72.0455794066
1 31.9233006624
2 -58.0981394854
3 0.343723173798
4 -3.71598046509e-09
5 0.11340322982
6 0.000101491907381
7 0.000935026972412
8 -46.8031386223
9 -58.0301612763
0 -59.7327082337
1 38.8677395605
2 -70.6003927936
3 0.473705067577
4 -4.76178306559e-09
5 0.0960889386051
6 0.000107717893413
7 0.000786184233302
8 -42.4686230142
9 -56.1717271284
0 -70.1297605684
1 34.9267334155
2 -74.9669130686
3 0.841101492027
4 -4.3089007837e-09
5 0.0930380532936
6 0.000103533932099
7 0.00125364654496
8 -40.2240399134
9 -59.084

/opt/conda/lib/python3.5/site-packages/scoop-0.7.2.0-py3.5.egg/scoop/fallbacks.py:46: RuntimeWarning: SCOOP was not started properly.
Be sure to start your program with the '-m scoop' parameter. You can find further information in the documentation.
Your map call has been replaced by the builtin serial Python map().


0.000771016165325
8 -36.5919387429
9 -55.8344549469
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}


In [ ]:
import sciunit
modelp
import pandas as pd
attributes = [ i.attrs for i in some_tuples[0] ]
rheobases = [ i.rheobase for i in some_tuples[0] ]
#modelsp = [model.__class__(p) for p in parameters]
models = some_tuples[1]
sc = pd.DataFrame(scores[0])
for j,i in enumerate(models):
        i.name = attributes[j]
sc

In [ ]:

pf = some_tuples[3]
#modelp.model_params
from neuronunit.tests import nsga

param_dict = modelp.model_params
trans_dict = nsga.get_trans_dict(param_dict)

In [ ]:
## The paretofront stores the best attributes according to the nsga.
vms=[]
for p in pf:
    p = nsga.individual_to_vm(p,trans_dict=trans_dict)
    vms.append(p.attrs)
    vms.append(p.error)
print(vms[-1])


In [ ]:
data = [ models[0].name ]
model_values0 = pd.DataFrame(data)        
model_values0


In [ ]:
rheobases[0]


In [ ]:

sc1 = pd.DataFrame(scores[1])
sc1


In [ ]:
rheobases[1]


In [ ]:
data=[ models[1].name ]
model_values1 = pd.DataFrame(data)        
model_values1

In [ ]:
models[1].name        

import pickle
import pandas as pd


try:
    ground_error = pickle.load(open('big_model_evaulated.pickle','rb'))
except:
    print('{0} it seems the error truth data does not yet exist, lets create it now '.format(str(False)))
    ground_error = list(futures.map(outils.func2map, ground_truth))
    pickle.dump(ground_error,open('big_model_evaulated.pickle','wb'))
#ground_error_nsga=list(zip(vmpop,pop,invalid_ind))
#pickle.dump(ground_error_nsga,open('nsga_evaulated.pickle','wb'))

#Get the differences between values obtained via brute force, and those obtained otherwise:
sum_errors = [ i[0] for i in ground_error ]
composite_errors = [ i[1] for i in ground_error ]
attrs = [ i[2] for i in ground_error ]
rheobase = [ i[3] for i in ground_error ]

indexs = [i for i,j in enumerate(sum_errors) if j==np.min(sum_errors) ][0]
indexc = [i for i,j in enumerate(composite_errors) if j==np.min(composite_errors) ][0]
#assert indexs == indexc
vmpop = some_tuples[0]
df_0 = pd.DataFrame([ (k,v,vmpop[0].attrs[k],float(v)-float(vmpop[0].attrs[k])) for k,v in ground_error[indexc][2].items() ])
df_1 = pd.DataFrame([ (k,v,vmpop[1].attrs[k],float(v)-float(vmpop[1].attrs[k])) for k,v in ground_error[indexc][2].items() ])



In [ ]:
#These are the differences in attributes found via brute force versus the genetic algorithm. For the top two candidates.

df_0

In [ ]:
df_1
